In [2]:
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, Math

import plotly.express as px
import matplotlib.pyplot as plt

### Parameters

In [3]:
cs = 343

f = 40e3  # carrier frequency
T = 1/f

wavelength = cs/f

In [4]:
def transducer(n, d, phi, f, t):
  w = 2 * np.pi * f
  wavelength = cs/f 
  
  return np.cos(w * t + n * 2 * np.pi / wavelength * d * np.sin(phi))

# Linear phase array beamforming
## 2 transducer

### Time signals

In [21]:
t = np.arange(-T, T, T/100)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)


@interact(phi=phi_slider, d=d_slider)
def plot2DBeamSignals(phi, d):
  E1 = transducer(0, d, phi, f, t)
  E2 = transducer(1, d, phi, f, t)
  E = E1 + E2
  
  # plt = px.line(x=t, y=[E1, E2, E])
  # display(plt)
  
  plt.figure(figsize=[10,4])
  plt.plot(t, E1, t, E2, t, E)
  plt.ylim(-2, 2)
  plt.xlim(-T, T)
  plt.xlabel('time in $s$')
  plt.ylabel('Magnitude')
  plt.legend(['Transducer 0', 'Transducer 1', 'Resulting Signal'], loc="upper right")


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle: ', max=3.14159265358…

### Polar signal

In [22]:
N = 2
vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/200)))

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

@interact(d=d_slider)
def plot2DBeamPolar(d):
  E = np.zeros([len(vphi), len(t)])
  for n in range(N):
    E = E + transducer(n, d, vphi, f, t)

  E_sum = np.array(np.sum(np.abs(E), 1)).flatten()
    
  plt.figure(figsize=[6, 6])
  plt.polar(np.array(vphi).flatten(), E_sum/np.max(E_sum))
  plt.show()


interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.00…

## N transducer
### Time signal

In [19]:
t = np.arange(-T, T, T/100)

phi_slider = widgets.FloatSlider(
    value=0,
    min=-np.pi,
    max=np.pi,
    step=0.01,
    description="Angle: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

@interact(phi=phi_slider, d=d_slider, N=N_slider)
def plot2DBeamSignals(phi, d, N):
  En = np.zeros([N, len(t)])
  
  plt.figure(figsize=[10, 4])
  for n in range(N):
    En[n] = transducer(n, d, phi, f, t)
    plt.plot(t, En[n])

  E = np.sum(En, 0)
  
  plt.plot(t, E)
  plt.ylim(-N, N)
  plt.xlim(-T, T)
  plt.xlabel('time in $s$')
  plt.ylabel('Magnitude')


interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='Angle: ', max=3.14159265358…

### Polar signal

In [26]:
vphi = np.transpose(np.matrix(np.arange(-np.pi, np.pi, np.pi/1000)))

d_slider = widgets.FloatSlider(
    value=wavelength/2,
    min=0,
    max=2 * wavelength,
    step=wavelength/100,
    description="Distance: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)

N_slider = widgets.IntSlider(
    value=2,
    min=1,
    max=20,
    step=1,
    description="Transducers: ",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
)

@interact(d=d_slider, N=N_slider)
def plot2DBeamPolar(d, N):
  E = np.zeros([len(vphi), len(t)])
  for n in range(N):
    E = E + transducer(n, d, vphi, f, t)

  E_sum = np.array(np.sum(np.abs(E), 1)).flatten()

  plt.figure(figsize=[6, 6])
  plt.polar(np.array(vphi).flatten(), E_sum/np.max(E_sum))
  plt.show()


interactive(children=(FloatSlider(value=0.0042875, continuous_update=False, description='Distance: ', max=0.01…